In [1]:
import xgboost as xgb
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [11]:
df=pd.read_csv('train_data_4000.csv')
#print(df)

In [12]:
#读取第2步骤卷积结果
dff=pd.read_csv('round_price_predict.csv')

In [13]:
df.insert(1,column='price_round',value=dff['0'])

In [14]:
#处理数据的时候不小心多了Unnamed: 0
print(dff.dtypes)
print(df)

0    float64
dtype: object
      Unnamed: 0  price_round   average  coordinate_x  coordinate_y  \
0              0    -0.948126 -1.837775      0.952906     -2.338354   
1              1    -0.521348 -1.898747      0.890253     -2.336596   
2              2     0.343674 -0.180653     -0.347061     -0.407445   
3              3     1.088772  0.686087     -0.199461     -0.287302   
4              4     0.841682  0.395272     -0.181054     -0.293427   
5              5     0.180311 -0.145374     -0.296314     -0.422673   
6              6     0.196196 -0.304227     -0.296314     -0.422673   
7              7    -0.397380  0.711540     -1.244624     -3.439777   
8              8     0.082522  0.197497     -0.374777     -0.334148   
9              9     0.410854  0.759090     -0.226759     -0.277829   
10            10    -0.419340  0.639735     -0.311342     -0.365063   
11            11     0.749826  0.381419     -0.267399     -0.340800   
12            12     2.843911  1.255205     -0.282

In [15]:
df.drop('Unnamed: 0',axis=1)

,price_round,average,coordinate_x,coordinate_y,decoration_condition,deed,elevator,facility0,facility1,facility2,...,apt,lift,district,rights,scale,structure,bath,kitchen,room,saloon
0,-0.948126,-1.837775,0.952906,-2.338354,-2.336298,1.483229,-0.836901,-0.618904,-0.297777,-0.476088,...,-0.618336,-0.442749,0.547417,0.285923,0.296690,1.429758,-0.465699,-0.040220,-0.156304,0.924514
1,-0.521348,-1.898747,0.890253,-2.336596,0.784828,-0.674205,-0.836901,-0.618904,-0.297777,-0.476088,...,-0.618336,-0.442749,0.547417,0.285923,2.284911,1.429758,1.530769,-0.040220,2.247519,0.924514
2,0.343674,-0.180653,-0.347061,-0.407445,-0.775735,1.483229,-0.836901,-0.618904,-0.297777,-0.476088,...,-0.618336,-0.442749,-0.001863,0.285923,-0.186393,-0.660088,-0.465699,-0.040220,1.045608,-0.679044
3,1.088772,0.686087,-0.199461,-0.287302,-0.775735,1.483229,1.194885,-0.618904,-0.297777,-0.476088,...,1.005616,-0.442749,-0.001863,0.285923,0.624103,-0.660088,-0.465699,-0.040220,-0.156304,0.924514
4,0.841682,0.395272,-0.181054,-0.293427,0.784828,-0.674205,1.194885,-0.618904,3.358214,2.100451,...,0.193640,1.717146,-0.001863,0.285923,0.725804,1.429758,-0.465699,-0.040220,1.045608,-0.679044
5,0.180311,-0.145374,-0.296314,-0.422673,-0.775735,1.483229,1.194885,-0.618904,3.358214,-0.476088,...,2.223580,1.717146,-0.001863,0.285923,-0.167204,-0.660088,-0.465699,-0.040220,-0.156304,-0.679044
6,0.196196,-0.304227,-0.296314,-0.422673,0.784828,1.483229,1.194885,1.615759,-0.297777,-0.476088,...,0.599628,1.717146,-0.001863,0.285923,0.242481,-0.660088,-0.465699,-0.040220,-0.156304,-0.679044
7,-0.397380,0.711540,-1.244624,-3.439777,-0.775735,1.483229,-0.836901,-0.618904,-0.297777,2.100451,...,0.193640,-0.442749,-0.001863,0.285923,-0.871440,1.429758,-0.465699,-0.040220,-0.156304,-0.679044
8,0.082522,0.197497,-0.374777,-0.334148,-0.775735,-0.674205,-0.836901,-0.618904,3.358214,2.100451,...,-0.618336,-0.442749,-0.001863,0.285923,-0.131704,-0.660088,-0.465699,-0.040220,1.045608,-0.679044
9,0.410854,0.759090,-0.226759,-0.277829,-0.775735,1.483229,-0.836901,-0.618904,-0.297777,-0.476088,...,0.193640,-0.442749,-0.001863,0.285923,-0.473988,-0.660088,-0.465699,-0.040220,1.045608,-0.679044


In [16]:
y=df['price'].copy()


In [17]:
X = df.drop(['price','Unnamed: 0'], axis = 1).copy()
print(X)

      price_round   average  coordinate_x  coordinate_y  decoration_condition  \
0       -0.948126 -1.837775      0.952906     -2.338354             -2.336298   
1       -0.521348 -1.898747      0.890253     -2.336596              0.784828   
2        0.343674 -0.180653     -0.347061     -0.407445             -0.775735   
3        1.088772  0.686087     -0.199461     -0.287302             -0.775735   
4        0.841682  0.395272     -0.181054     -0.293427              0.784828   
5        0.180311 -0.145374     -0.296314     -0.422673             -0.775735   
6        0.196196 -0.304227     -0.296314     -0.422673              0.784828   
7       -0.397380  0.711540     -1.244624     -3.439777             -0.775735   
8        0.082522  0.197497     -0.374777     -0.334148             -0.775735   
9        0.410854  0.759090     -0.226759     -0.277829             -0.775735   
10      -0.419340  0.639735     -0.311342     -0.365063             -0.775735   
11       0.749826  0.381419 

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [19]:
clf_xgb=xgb.XGBRegressor(objective='reg:squarederror',max_depth=50,n_jobs=1)

In [20]:
xgb_model=clf_xgb.fit(X_train,y_train,verbose = True,
            early_stopping_rounds = 5,
                      
            eval_metric = 'rmse',
            eval_set = [(X_test, y_test)])

[0]	validation_0-rmse:4606860.00000
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:3521025.25000
[2]	validation_0-rmse:2817933.75000
[3]	validation_0-rmse:2343491.75000
[4]	validation_0-rmse:1989347.50000
[5]	validation_0-rmse:1796876.50000
[6]	validation_0-rmse:1620902.87500
[7]	validation_0-rmse:1496636.12500
[8]	validation_0-rmse:1410178.12500
[9]	validation_0-rmse:1330504.87500
[10]	validation_0-rmse:1269320.75000
[11]	validation_0-rmse:1223610.12500
[12]	validation_0-rmse:1188010.50000
[13]	validation_0-rmse:1159591.75000
[14]	validation_0-rmse:1136750.50000
[15]	validation_0-rmse:1119109.87500
[16]	validation_0-rmse:1105049.00000
[17]	validation_0-rmse:1094172.00000
[18]	validation_0-rmse:1085323.62500
[19]	validation_0-rmse:1077684.62500
[20]	validation_0-rmse:1071570.50000
[21]	validation_0-rmse:1066238.75000
[22]	validation_0-rmse:1061820.12500
[23]	validation_0-rmse:1058210.75000
[24]	validation_0-rmse:1055132.75000
[25]	validation_0-rms

In [21]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
predictions = xgb_model.predict(X_test)
actuals = y_test
mean_absolute_error(actuals,predictions)

209739.77235264736

In [22]:
actuals_n=0
n=0
for i in actuals:
    if(i-predictions[n]<10000):
        actuals_n=actuals_n+1
    n=n+1
print(actuals_n,len(predictions))
print('err:%f4'%(actuals_n/len(predictions)))

550 1001
err:0.5494514
